In [1]:
import pandas as pd
import os.path
import openpyxl as xl
import channels
import utils
import re
import json
import facebook_business.api as fbapi
import facebook_business.adobjects.adaccount as a
import facebook_business.session as s


In [2]:
# Retrieve configuration stuff
config_file = os.path.join(os.getcwd(),r"facebook_business\config.json")
config_open = open(config_file)
config = json.load(config_open)

In [3]:
# Get old files
# 1. create files for all time all accounts up until August 10
# 2. API call finds the latest date in the files and updates them with a new file
# 2a. If nothing is found, the API will default to August 10, 2024.
hist_soc_data = utils.data_framer(source=utils.assumptions()['source']
                                ,files=utils.file_gatherer(utils.assumptions()['source'],criteria=".*[Ss]ocial.*")
                                ,sheets=None
                                ,skiprows=None)


>>> 2024-08-13 11:35:33.493611 ]|[ Assumptions loaded.
>>> 2024-08-13 11:35:33.493611 ]|[ Source is N:\Marketing Analytics\Brand Analysis\04 - Dashboards\Brand Spend Data Model\Source Data
>>> 2024-08-13 11:35:33.516748 ]|[ Target is N:\Marketing Analytics\Brand Analysis\04 - Dashboards\Brand Spend Data Model\Target Data
>>> 2024-08-13 11:35:33.593807 ]|[ Assumptions loaded.
>>> 2024-08-13 11:35:33.610504 ]|[ Source is N:\Marketing Analytics\Brand Analysis\04 - Dashboards\Brand Spend Data Model\Source Data
>>> 2024-08-13 11:35:33.610504 ]|[ Target is N:\Marketing Analytics\Brand Analysis\04 - Dashboards\Brand Spend Data Model\Target Data
>>> 2024-08-13 11:35:33.637851 ]|[ Files gathered based on .*[Ss]ocial.* applied to N:\Marketing Analytics\Brand Analysis\04 - Dashboards\Brand Spend Data Model\Source Data:
>>> 2024-08-13 11:35:33.643642 ]|[ TARGET FILE NAME: C1B-Paid-Social-NDC1 (2).xlsx
>>> 2024-08-13 11:35:33.659576 ]|[ TARGET FILE NAME: C1B-Paid-Social-NDC1 old.xlsx
>>> 2024-08-13

c:\Program Files\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


>>> 2024-08-13 11:35:38.678501 ]|[ DF from file C1B-Paid-Social-NDC1 (2).xlsx has len 23051
Concat complete
>>> 2024-08-13 11:35:49.306809 ]|[ DF from file C1B-Paid-Social-NDC1 old.xlsx has len 50966
Concat complete


c:\Program Files\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


>>> 2024-08-13 11:35:51.137387 ]|[ DF from file C1B-Paid-Social-NDC1-2023dec.xlsx has len 1787
Concat complete
>>> 2024-08-13 11:35:51.873077 ]|[ DF from file C1B-Social-Apr-1-2024-to-Apr-30-2024.csv has len 3066
Concat complete
>>> 2024-08-13 11:35:52.013288 ]|[ DF from file C1B-Social-Feb-1-2024-to-Feb-29-2024.csv has len 398
Concat complete
>>> 2024-08-13 11:35:52.295594 ]|[ DF from file C1B-Social-Jan-1-2024-to-Jan-31-2024.csv has len 2503
Concat complete
>>> 2024-08-13 11:35:52.453185 ]|[ DF from file C1B-Social-May-1-2024-to-May-31-2024.csv has len 468
Concat complete
>>> 2024-08-13 11:35:52.501425 ]|[ Dataset has len of 82239


In [4]:
# Determine when the API should start retrieving data
latest_date = hist_soc_data['Day'].max()
if latest_date > '2024-08-10':
    start_date = latest_date
else:
     start_date = '2024-08-10'

In [5]:

session = s.FacebookSession(app_id=config['app_id'], app_secret=config['app_secret'], access_token=config['access_token'], proxies=None, timeout=None, debug=False)

In [6]:

connection = fbapi.FacebookAdsApi().init(access_token="EAADTjffqlZBQBO3HXaZC55FdvL7hZCVemNjsfXWCCe9vaPKWq00nesL3itPng74lj5SIjWuncZBXflH6XR423AvvwV9AuyvkPPG8iRK1KNPgCI5STF8vHLlgieHXr9vmRXXMY0rLBGHzHAecE4lWZBLya5hA8DTXTmLeDGLtyLTYVKiv9EZB20gTkk"
                                             , session=session)


TypeError: FacebookAdsApi.__init__() missing 1 required positional argument: 'session'

In [ ]:

# Access Token Sorter
# if len(config['access_token']) > 1:
#     if access_checker(access_token=config['access_token']):
#         print(f"{time.time()} >>> Option 1: Long-term access token available and valid.",file=open(log,'a'))
#         access_token = self.access_token
#         days = 60
#     else:
#         print(f"{time.time()} >>> Option 2: Long-term access token available, but invalid.",file=open(log,'a'))
#         new_access_token = input("Please enter a new access token.")
#         response = self.get_fb_token(access_token=new_access_token)
#         access_token = response.json()['access_token']
#         self.token_writer(access_token=access_token)
#         days = response.json()['expires_in'] / 60 / 60 / 24
# else:
#     if self.access_checker(access_token=self.initial_access_token):
#         print(f"{time.time()} >>> Option 3: No long-term access token, initial access token used to create long-term access token.",file=open(log,'a'))
#         response = self.get_fb_token(access_token=self.initial_access_token)
#         access_token = response.json()['access_token']
#         self.token_writer(access_token=access_token)
#         days = response.json()['expires_in'] / 60 / 60 / 24
#     else:
#         print(f"{time.time()} >>> option 4: No valid access tokens availble, please enter new token.",file=open(log,'a'))
#         new_access_token = input("Please enter a new access token.")
#         response = self.get_fb_token(access_token=new_access_token)
#         access_token = response.json()['access_token']
#         self.token_writer(access_token=access_token)
#         days = response.json()['expires_in'] / 60 / 60 / 24

# print(f"This access_token expires in {str(days)} days.",file=open(log,'a'))

Did i call her out on her bullshit in the middle of a lie?

What in the fuck is this DM test?
What does it have to do with brand?
